In [5]:
import pandas as pd

In [14]:
whichContestsDF = pd.read_csv("data/Durations.csv").drop(columns=['Unnamed: 0']).set_index("ContestId")
whichContestsDF = whichContestsDF[whichContestsDF['Duration']>240]
contests = pd.read_csv("data/WorkedData.csv").set_index("ContestId")[['Success']]
chunkMap = pd.read_csv("data/ChunkMap.csv").set_index("ContestId")

In [35]:
jacksonDF = pd.merge(whichContestsDF, contests, on='ContestId', how='left')
jacksonDF = pd.merge(jacksonDF, chunkMap, on='ContestId', how='left')
jacksonDF = jacksonDF.dropna().reset_index()#.drop(columns=['Unnamed: 0'])

In [41]:
from ipywidgets import IntProgress
from IPython.display import display

In [65]:
f = IntProgress(min=0, max=48)
display(f)
chunks = []
for chunkName in jacksonDF['Chunk'].unique()[:]:
    chunkDF = pd.read_csv('data/Chunks/'+chunkName+".csv").drop(columns=['Unnamed: 0'])
    chunkDF = chunkDF[chunkDF['ContestId'].isin(jacksonDF['ContestId'].unique())]
    chunks.append(chunkDF)
    f.value+=1
allChunksDF = pd.concat(chunks)

IntProgress(value=0, max=48)

In [68]:
toMerge = []
cids = jacksonDF['ContestId'].unique()
f = IntProgress(min=0, max=len(cids))
display(f)
for cid in cids:
    justContest = allChunksDF[allChunksDF['ContestId']== cid]
    justContest = justContest.sort_values(by='SecondsRemaining', ascending = False)
    justContest['Summed'] = justContest['Entries'].cumsum()
    justContest['MinutesRemaining'] = justContest['SecondsRemaining']
    toMerge.append(justContest[['ContestId', 'Summed', 'MinutesRemaining']])
    f.value+=1

IntProgress(value=0, max=259896)

In [70]:
mergedDF = pd.merge(jacksonDF, pd.concat(toMerge), on='ContestId', how='outer')

In [74]:
forJ=mergedDF.dropna()

In [75]:
forJ.to_csv('data/JacksonData_01_29.csv')